The general idea is to 

In [2]:
import sympy as sp
import networkx as nx
sp.init_printing()

def check_bad_terms(polynomial, base_variables, dependence_graph, terms_with_generated_relations):
    """
    Args:
        polynomial : 
        base_variables : 
        dependence_graph : 
        terms_with_generated_relations : 
    Returns:
        new_update_relations_needed : List of sets of sympy symbols
    """
    new_update_relations_needed = []
    for mono in polynomial.monoms():
        variables_in_mono = {base_variables[i] for i, e in enumerate(mono) if e != 0}
        # Find the graph of variable dependencies for this particular monomial.
        mono_dependence_graph = dependence_graph.subgraph(variables_in_mono)
        if len(mono_dependence_graph.edges) == 0:
            # All variables in this term are independent of each other.
            pass
        else:
            connected_components = nx.connected_components(mono_dependence_graph)
            for component in connected_components:
                if len(component) > 1 and not any([component == group for group in terms_with_generated_relations]) and component not in new_update_relations_needed:
                    new_update_relations_needed.append(component)
    return new_update_relations_needed

def generate_new_update_relations(need_update_relations, base_variable_updates, base_variables):
    """
    Args:
        need_update_relations: list of frozensets of variables
        base_variable_updates: 
        base_variables: 
    Returns:
        new_update_relations
    """
    new_update_relations = []
    for new_relation_vars in need_update_relations:
        new_relation = 1
        for var in new_relation_vars:
            new_relation *= base_variable_updates[var]
        new_update_relations.append(sp.poly(new_relation, base_variables))
    return new_update_relations

def iterate_relations(relations_to_check, base_variables, variable_dependence_graph, terms_with_generated_relations):
    """
    Args:
        relations_to_check: 
        base_variables: 
        variable_dependence_graph: 
        terms_with_generated_relations: 
    Returns:
        new_relations_to_check
        terms_with_generated_relations
    """
    need_update_relations = []
    # Given a list of relations to check, generate a new set of relations to check.
    for relation in relations_to_check:
        temp = check_bad_terms(relation, base_variables, variable_dependence_graph, terms_with_generated_relations)
        need_update_relations += temp
    print(need_update_relations)
    # Eliminate redundant stuff
    need_update_relations = {frozenset(s) for s in need_update_relations}
    need_update_relations = [set(s) for s in need_update_relations]
    
    # Generate new relations based off what we have in need_update_relations
    new_relations_to_check = generate_new_update_relations(need_update_relations, base_variable_updates, base_variables)
    terms_with_generated_relations += need_update_relations
    return new_relations_to_check, terms_with_generated_relations
    

# Declare variables at time "t"
xt = sp.Symbol("x_t")
yt = sp.Symbol("y_t")
vt = sp.Symbol("v_t")
wvt = sp.Symbol("w_{v_t}")
wthetat = sp.Symbol("w_{theta_t}")
thetat = sp.Symbol("theta_t")
cos_thetat = sp.cos(thetat)
sin_thetat = sp.sin(thetat)
sin_thetat = sp.sin(thetat)
cos_thetat = sp.cos(thetat)
sin_wthetat = sp.sin(wthetat)
cos_wthetat = sp.cos(wthetat)

# Create a list of variables at time "t"
# Indicies for the variables:
# 0 : xt
# 1 : yt
# 2 : vt
# 3 : sin_thetat
# 4 : cos_thetat
# 5 : sin_wthetat
# 6 : cos_wthetat
base_variables = [xt, yt, vt, sin_thetat, cos_thetat, sin_wthetat, cos_wthetat]
variable_dependence_graph = nx.Graph()
variable_dependence_graph.add_nodes_from(base_variables)
variable_dependence_graph.add_edges_from([(xt, yt), (xt, vt), (yt, vt), (xt, sin_thetat), (xt, cos_thetat), (yt, sin_thetat), (yt, cos_thetat)])

# Expressions we have update relations for.
terms_with_generated_relations = [set([xt, sin_thetat]), set([xt, cos_thetat]), set([yt, sin_thetat]), set([yt, cos_thetat])]

# Update Relations for Base Variables
xtplus1 = xt + vt * cos_thetat
ytplus1 = yt + vt * sin_thetat
vtplus1 = vt + wvt
sin_thetatplus1 = sp.expand_trig(sp.sin(thetat + wthetat))
cos_thetatplus1 = sp.expand_trig(sp.cos(thetat + wthetat))
base_variable_updates = {}
base_variable_updates[xt] = xtplus1
base_variable_updates[yt] = ytplus1
base_variable_updates[vt] = vtplus1
base_variable_updates[sin_thetat] = sin_thetatplus1
base_variable_updates[cos_thetat] = cos_thetatplus1

# Construct the expression for E[x_{t+1} * y_{t+1}] in terms of expressions at time t
cross = sp.poly(xtplus1 * ytplus1, base_variables)
terms_with_generated_relations += [set([xt, yt])]

i = 0
relations_to_check = [cross]

while relations_to_check:
    relations_to_check, terms_with_generated_relations = iterate_relations(relations_to_check, base_variables, variable_dependence_graph, terms_with_generated_relations)

xt_sq = sp.poly(xtplus1 ** 2, base_variables)


[{v_t, x_t, sin(theta_t)}, {v_t, y_t, cos(theta_t)}]
[{v_t, x_t, cos(theta_t)}, {v_t, y_t, sin(theta_t)}]
[]
[{x_t, sin(theta_t)}, {x_t, cos(theta_t)}, {y_t, sin(theta_t)}, {y_t, cos(theta_t)}, {y_t, x_t}, {v_t, x_t, sin(theta_t)}, {v_t, y_t, cos(theta_t)}, {v_t, x_t, cos(theta_t)}, {v_t, y_t, sin(theta_t)}]


Detect when the "invalid pair" is the term we are updating
We need to dynamically change the list of bad_var_pairs as we derive new update relations (maybe we should explicitly keep track of the update relations that we actually have instead)....

[x_t, y_t]
